<h1>Deep Encode Group 2 Lableing</h1>

labeling pipeline:
1. define minimum vmaf score

2. obtain meta info of video and store in some useful format

3. split into scenes (nominally mp4, but actually y4m)

4. rename to y4m

5. **compress**

6. **calc vmaf compared to original scene**

**bold**: repeat until acceptable, subject to compression strategy


In [51]:
import os
import subprocess
import pandas as pd
import numpy as np
import re
import math


In [49]:
### PARAMETERS ###

# define minimum acceptable vmaf
minimum_acceptable_vmaf = 98.0

# bitrates from 1 to 64 MB/s possible
bitrate_candidates = np.array(range(1, 65))


### LOCATIONS ###

# relative path to input video
input_vid_loc='_testing/factory_1080p30.y4m'

# relative path to split uncompressed scenes
orig_scenes_loc='_testing/orig_scenes'


### INPUT VIDEO META DATA ###

input_vid_res = '1920x1080'

input_vid_fps = '30'

input_vid_pix_fmt = 'yuv420p'

In [47]:
# HELPER METHODS

def extract_vmaf(result_string):
    # Extract the number using regex
    match = re.search(r"VMAF score: (\d+\.\d+)", result_string)

    # Check if a match is found and get the number
    if match:
        vmaf_score = float(match.group(1))
        return vmaf_score 
    else:
        print("VMAF score not found in the string")

def delete_encode(path):
    if os.path.isfile(path):
        # Delete the file
        os.remove(path)
        print(f"The file at {path} has been deleted.")
    else:
        print(f"No file found at {path}.")

In [4]:
! ffprobe -v error -show_streams {input_vid_loc}

[STREAM]
index=0
codec_name=rawvideo
codec_long_name=raw video
profile=unknown
codec_type=video
codec_tag_string=I420
codec_tag=0x30323449
width=1920
height=1080
coded_width=1920
coded_height=1080
closed_captions=0
film_grain=0
has_b_frames=0
sample_aspect_ratio=1:1
display_aspect_ratio=16:9
pix_fmt=yuv420p
level=-99
color_range=unknown
color_space=unknown
color_transfer=unknown
color_primaries=unknown
chroma_location=unspecified
field_order=progressive
refs=1
id=N/A
r_frame_rate=30/1
avg_frame_rate=30/1
time_base=1/30
start_pts=0
start_time=0.000000
duration_ts=1339
duration=44.633333
bit_rate=N/A
max_bit_rate=N/A
bits_per_raw_sample=N/A
nb_frames=N/A
nb_read_frames=N/A
nb_read_packets=N/A
DISPOSITION:default=0
DISPOSITION:dub=0
DISPOSITION:original=0
DISPOSITION:comment=0
DISPOSITION:lyrics=0
DISPOSITION:karaoke=0
DISPOSITION:forced=0
DISPOSITION:hearing_impaired=0
DISPOSITION:visual_impaired=0
DISPOSITION:clean_effects=0
DISPOSITION:attached_pic=0
DISPOSITION:timed_thumbnails=0
DISP

In [7]:
# split scenes bases on content detection
# in this command no additional info about the input video is provided

! scenedetect -i {input_vid_loc} -o {orig_scenes_loc} detect-content split-video -a "-map 0 -c:v copy -c:a copy -f rawvideo -vcodec rawvideo"

[PySceneDetect] PySceneDetect v0.6.1
[PySceneDetect] Output directory set:
  /Users/leon/Desktop/uni/23ss/awt/code/Deep-Encode-TUB/labeling/_testing/orig_scenes
[PySceneDetect] ffmpeg arguments: -map 0 -c:v copy -c:a copy -f rawvideo -vcodec rawvideo
[PySceneDetect] Downscale factor set to 7, effective resolution: 274 x 154
[PySceneDetect] Detecting scenes...                                             
Detected: 18 | Progress: 100%|█████████| 1339/1339 [00:03<00:00, 344.08frames/s]
[PySceneDetect] Processed 1339 frames in 3.9 seconds (average 341.65 FPS).
[PySceneDetect] Detected 19 scenes, average shot length 2.3 seconds.
[PySceneDetect] Comma-separated timecode list:
  00:00:03.567,00:00:09.133,00:00:09.733,00:00:10.333,00:00:11.100,00:00:11.700,00:00:15.500,00:00:17.167,00:00:17.900,00:00:18.500,00:00:19.100,00:00:21.667,00:00:23.033,00:00:23.800,00:00:24.400,00:00:28.367,00:00:34.100,00:00:39.167
[PySceneDetect] Splitting input video using ffmpeg, output path template:
  /Users/le

In [8]:
# rename to y4m to correctly refelct data format

def change_file_suffix(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.mp4'):
            new_filename = os.path.splitext(filename)[0] + '.y4m'
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_filename)
            os.rename(old_path, new_path)
            print(f'Renamed {filename} to {new_filename}')

change_file_suffix(orig_scenes_loc)


Renamed factory_1080p30-Scene-013.mp4 to factory_1080p30-Scene-013.y4m
Renamed factory_1080p30-Scene-007.mp4 to factory_1080p30-Scene-007.y4m
Renamed factory_1080p30-Scene-006.mp4 to factory_1080p30-Scene-006.y4m
Renamed factory_1080p30-Scene-012.mp4 to factory_1080p30-Scene-012.y4m
Renamed factory_1080p30-Scene-004.mp4 to factory_1080p30-Scene-004.y4m
Renamed factory_1080p30-Scene-010.mp4 to factory_1080p30-Scene-010.y4m
Renamed factory_1080p30-Scene-011.mp4 to factory_1080p30-Scene-011.y4m
Renamed factory_1080p30-Scene-005.mp4 to factory_1080p30-Scene-005.y4m
Renamed factory_1080p30-Scene-001.mp4 to factory_1080p30-Scene-001.y4m
Renamed factory_1080p30-Scene-015.mp4 to factory_1080p30-Scene-015.y4m
Renamed factory_1080p30-Scene-014.mp4 to factory_1080p30-Scene-014.y4m
Renamed factory_1080p30-Scene-016.mp4 to factory_1080p30-Scene-016.y4m
Renamed factory_1080p30-Scene-002.mp4 to factory_1080p30-Scene-002.y4m
Renamed factory_1080p30-Scene-003.mp4 to factory_1080p30-Scene-003.y4m
Rename

<h2>Encoding strategy:</h2>

We use ffmpeg's libx264 at preset medium (default).

Rate Control Mode is **Two Pass average bitrate (ABR)** -b:v, which results in a guaranteed average bitrate over the scene. At the same time the encoder is still able to adapt to different 'complexities' within the scene. Results are same as in 'capped CRF'(see: https://slhck.info/video/2017/03/01/rate-control.html). We will not set -maxrate or -bufsize in this mode.

(Note, bc otherwise you will get confused:  -maxrate in combination with crf means the maximum bitrate of the entire video that is acceptable.
-maxrate in association with -b:v is the maximum acceptale "abweichung" from average bitrate!)

~~We aim for an encoing that at highest settings encodes at "-crf 17". This constant rate factor is also recommended (by https://trac.ffmpeg.org/wiki/Encode/H.264) for visually lossless compression at highest compression rate.~~

~~This constant rate factor (crf), however, will be limited by "-maxrate" which specifies a maximum bitrate. (We will set bufsize to maxrate*2). According to documentation (link above), this setting does not strictly guarantee maxrate as bitrate, but we found it still complying pretty well, while still always aiming for an optimal result.~~


In [52]:
# test encode

def test_encode(directory):
    for filename in os.listdir(directory):

        #new dir per scene/filename
        new_dir_name=os.path.splitext(filename)[0]
        dir_path = os.path.join(directory, new_dir_name)
        os.makedirs(dir_path)

        #do qp0 encode
        qp0_filename = f'{dir_path}/qp0_{new_dir_name}.mp4'
        qp0_command = f'ffmpeg -v error -f rawvideo -vcodec rawvideo -s {input_vid_res} -r {input_vid_fps} -pix_fmt {input_vid_pix_fmt} -i {orig_scenes_loc}/{filename} -c:v libx264 -qp 0 {qp0_filename}'
        qp0_result = subprocess.run(qp0_command, capture_output=True, text=True, shell=True)
        print('QP0 RESULT: ', qp0_result)

        #init binar search
        low = 0
        high = len(bitrate_candidates) - 1
        mid = 0
 
        #iterations for binary search
        iterations=math.floor(math.log2(len(bitrate_candidates)))

        #BINARY SEARCH OVER BITRATES
        while range(iterations):

            mid = (high + low) // 2

            current_bitrate=bitrate_candidates[mid]
            print(f'CURRENT BITRATE: {current_bitrate}')

            #do encode
            encoded_filename = f'{dir_path}/{current_bitrate}M_{new_dir_name}.mp4'
            encode_command = f'ffmpeg -v error -f rawvideo -vcodec rawvideo -s {input_vid_res} -r {input_vid_fps} -pix_fmt {input_vid_pix_fmt} -i {orig_scenes_loc}/{filename} -c:v libx264 -b:v {current_bitrate}M -pass 1 -f null /dev/null &&    \
                                ffmpeg -v error -f rawvideo -vcodec rawvideo -s {input_vid_res} -r {input_vid_fps} -pix_fmt {input_vid_pix_fmt} -i {orig_scenes_loc}/{filename} -c:v libx264 -b:v {current_bitrate}M -pass 2 {encoded_filename}'
            encode_result = subprocess.run(encode_command, capture_output=True, text=True, shell=True)
            print('ENCODE RESULT: ', encode_result)

            # calc vmaf
            vmaf_command = f'ffmpeg -i {encoded_filename} -i {qp0_filename} -filter_complex libvmaf -f null -'
            vmaf_result = subprocess.run(vmaf_command, capture_output=True, text=True, shell=True)
            print('VMAF RESULT: ', vmaf_result)

            vmaf_score=extract_vmaf(str(vmaf_result))
            print('VMAF SCORE: ', vmaf_score)
            
            #delete encode
            delete_encode(encoded_filename)

            # final iteration
            if low == high: # TODO check if always converges
                
                if vmaf_score > last_upper_candidate_vmaf or vmaf_score < minimum_acceptable_vmaf:
                    print(f'CONVERGED: LAST UPPERCANDIDATE IS OPTIMAL. current vmaf_score: {vmaf_score} vs last_upper_candidate_vmaf: {last_upper_candidate_vmaf}')
                    print(f'FINAL BITRATE LABEL FOR FILE {filename}: {last_upper_candidate}')
                
                elif vmaf_score < last_upper_candidate_vmaf and vmaf_score > minimum_acceptable_vmaf:
                    print(f'CONVERGED: CURRENT VMAF IS OPTIMAL. current vmaf_score: {vmaf_score} vs last_upper_candidate_vmaf: {last_upper_candidate_vmaf}')
                    print(f'FINAL BITRATE LABEL FOR FILE {filename}: {current_bitrate}')

                else:
                    print('ERROR DID NOT FIND OPTIMAL BITRATE')

                #delete qp0 encode
                delete_encode(qp0_filename)

                #delete scene dir
                os.rmdir(dir_path)

                break
 
            elif vmaf_score < minimum_acceptable_vmaf:
                low = mid + 1
        
            elif vmaf_score > minimum_acceptable_vmaf:
                high = mid - 1
                last_upper_candidate=current_bitrate
                last_upper_candidate_vmaf=vmaf_score
            
            else:
                print('ERROR DID NOT FIND OPTIMAL BITRATE')
                
            
test_encode(orig_scenes_loc)



QP0 RESULT:  CompletedProcess(args='ffmpeg -v error -f rawvideo -vcodec rawvideo -s 1920x1080 -r 30 -pix_fmt yuv420p -i _testing/orig_scenes/factory_1080p30-Scene-019.y4m -c:v libx264 -qp 0 _testing/orig_scenes/factory_1080p30-Scene-019/qp0_factory_1080p30-Scene-019.mp4', returncode=0, stdout='', stderr='')
CURRENT BITRATE: 32
ENCODE RESULT:  CompletedProcess(args='ffmpeg -v error -f rawvideo -vcodec rawvideo -s 1920x1080 -r 30 -pix_fmt yuv420p -i _testing/orig_scenes/factory_1080p30-Scene-019.y4m -c:v libx264 -b:v 32M -pass 1 -f null /dev/null &&                                    ffmpeg -v error -f rawvideo -vcodec rawvideo -s 1920x1080 -r 30 -pix_fmt yuv420p -i _testing/orig_scenes/factory_1080p30-Scene-019.y4m -c:v libx264 -b:v 32M -pass 2 _testing/orig_scenes/factory_1080p30-Scene-019/32M_factory_1080p30-Scene-019.mp4', returncode=0, stdout='', stderr='')
VMAF RESULT:  CompletedProcess(args='ffmpeg -i _testing/orig_scenes/factory_1080p30-Scene-019/32M_factory_1080p30-Scene-019.mp4

KeyboardInterrupt: 